# Amazon Bedrock Activation Workshop Chapter1: プロンプトエンジニアリング

本チャプターでは、Claude v2 を使った、プロンプトエンジニアリングについて説明します。  
システム設計を行うプロンプトや、宣伝メールを生成するプロンプトの例をもとに、実際のアプリケーション開発を想定した解説を行います。

## 事前準備

In [ ]:
from typing import List, Dict
from sagemaker import Session
import boto3
from boto3.session import Session
from langchain.chat_models import BedrockChat
from langchain.schema import HumanMessage
from langchain.chains import *
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

from langchain.prompts.chat import *

In [ ]:
# Need update to GNU C++ compiler version
!apt-get update && apt-get install -y build-essential --quiet
!pip install shap --quiet
# Installs dependencies reuqired for chromadb memory vectordb and embedding library
!pip install chromadb tiktoken langchain --quiet

In [ ]:
from langchain.llms import Bedrock
from langchain.chat_models import BedrockChat
from langchain.schema import HumanMessage
import json

bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

In [ ]:
def invoke_claude(text, max_tokens_to_sample=1000): 

    body = json.dumps({
        "prompt": f"\n\nHuman:{text}\n\nAssistant: ",
        "max_tokens_to_sample": max_tokens_to_sample,
        "temperature": 0.1,
        "top_p": 0.9,
    })

    modelId = 'anthropic.claude-v2'
    accept = 'application/json'
    contentType = 'application/json'

    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())

    return response_body.get('completion')[1:]

In [ ]:
chat = BedrockChat(
    client=bedrock_runtime,
    model_id="anthropic.claude-v2",
    model_kwargs={
        "temperature":0.0,
        "max_tokens_to_sample": 2000},
)

## 基本的なアプリケーションの作成

名前の候補を考えてくれるアプリケーションを作成してみましょう。  
まずはアプリケーションで利用する LLM のライブラリの名前の候補を5つ Claude に考えてもらいましょう。  
以下のようなプロンプトを与えます。  

In [ ]:
prompt = "LLMのプロンプトエンジニアリングのためにテンプレートを作ることができるライブラリの名前の候補を5つ考えてください。"
print(invoke_claude(prompt))

それらしい回答が得られたのではないかと思います。  
出力結果をプログラムで利用しやすくするため、余計な部分を除いてリストのみの出力を強制させます。  

In [ ]:
prompt = "LLMのプロンプトエンジニアリングのためにテンプレートを作ることができるライブラリの名前の候補を5つ考えてください。ただし、リストのみを出力してください。"
print(invoke_claude(prompt))

リスト形式での回答が得られました。  

### プロンプトテンプレートの活用

次に、LLM のライブラリ以外にも名前を考えてもらえるようにしましょう。  
プロンプトのテンプレートを作り、文字列の format 関数でテンプレートの一部を置き換えられるようにします。  
（LLM とは関係なく Python の format 関数ですので、f文字列の利用も可能です。）

In [ ]:
prompt_template = "{something}の名前の候補を5つ考えてください。ただし、リストのみを出力してください。"

In [ ]:
prompt = prompt_template.format(something="肉汁が溢れるハンバーグと焼きたてのバンズからなる美味しいハンバーガー")
print('Prompt:', prompt)
print('Output:', invoke_claude(prompt))

In [ ]:
prompt = prompt_template.format(something="お年寄りにも使いやすいチャットシステム")
print('Prompt:', prompt)
print('Output:', invoke_claude(prompt))

より多くの情報をプロンプトに入力することもできます。  
システムアーキテクトとして Web サービスのアーキテクチャ設計を行うものを作ってみましょう。

In [ ]:
prompt_template = """
以下のようなシステムのアーキテクチャを考えてください。  
システム概要: {about}
システム規模: {scale}  
機能: {features}
"""

prompt = prompt_template.format(
    about="衣料品を販売するECサイト", 
    scale="ピーク時には毎分10000リクエストに対応できる必要があります。また、グローバルに利用可能である必要があります。また、応答性が高く高速である必要があります。", 
    features="""次の3つのページを含んでいる必要があります。
    1.製品について解説したランディングページ。 2. 会社概要を説明するページ 3. 採用情報ページ
    """
)

print('Prompt:', prompt)
print('Output:', invoke_claude(prompt))

以上のような回答でも十分かもしれませんが、より分かりやすい回答を引き出すようにプロンプトを工夫します。  
「あなたは熟達したシステムアーキテクトです」のように、役割を含めてプロンプトを作成します。(Role Prompting)

In [ ]:
prompt_template = """
あなたは熟達したシステムアーキテクトです。初心者にとっても専門用語を噛み砕いて分かりやすい説明をすることで有名です。
以下のようなシステムのアーキテクチャを考えてください。 
システム概要: {about}
システム規模: {scale}  
機能: {features}
"""

prompt = prompt_template.format(
    about="衣料品を販売するECサイト", 
    scale="ピーク時には毎分10000リクエストに対応できる必要があります。また、グローバルに利用可能である必要があります。また、応答性が高く高速である必要があります。", 
    features="""次の3つのページを含んでいる必要があります。
    1.製品について解説したランディングページ。 2. 会社概要を説明するページ 3. 採用情報ページ
    """
)

print('Prompt:', prompt)
print('Output:', invoke_claude(prompt))

初心者にとっても分かりやすい説明が出力されました。  
さらに、プロンプトを構造化することで、より良い出力結果が得られやすいようにします。  
入力する情報を<></>タグで囲い、構造化することで、より入力に即した結果が返りやすくなります。

In [ ]:
prompt_template = """
<Instruction>
あなたは熟達したシステムアーキテクトです。初心者にとっても専門用語を噛み砕いて分かりやすい説明をすることで有名です。
システムの概要、規模、機能の情報をもとにお客様の要求を分析し、セキュリティ・パフォーマンス・運用性・信用性・コスト最適といった観点でアーキテクチャを考案してください。
また、高校生にもわかるように、専門用語を噛み砕いて説明してください。
</Instruction> 
<Requirements>
    <About>
        {about}
    </About>
    <Scale>
        {scale}
    </Scale>
    <Feature>
        {features}
    </Feature>
</Requirements>
"""

In [ ]:
prompt = prompt_template.format(
    about="ECサイトのためのウェブサイト", 
    scale="ピーク時には毎秒10000リクエストに対応できる必要があります。また、グローバルに利用可能である必要があります。また、応答性が高く高速である必要があります。", 
    features="""次の3つのページを含んでいる必要があります。
    1.製品について解説したランディングページ。 2. 会社概要を説明するページ 3. 採用情報ページ
    """
)

print('Prompt:', prompt)
print('Output:', invoke_claude(prompt))

いかがでしょうか、先ほどよりもさらに洗練され、分かりやすい説明が出力されたのではないでしょうか。  
このように、プロンプトを工夫することで、基盤モデルの出力を向上させることができます。  

### 練習1

あなたは、マーケティングチームの一員です。  
お客様ごとにカスタマイズされたマーケティングメールを大規模に提供できるように取り組んでいます。  
そこで、基盤モデルでマーケティングメールを自動生成させたいと考えています。

メールにはユーザーに関する以下の情報を含めることとします。（データベースから取得する想定）
* 名前 (name)
* 年齢 (age)
* 興味のあるもの (文字列のリスト)

また、製品の情報として、以下の情報を持った辞書型変数も利用することが可能です。
* 商品名 
* 詳細

これらの情報をもとに、お客様ごとに最適化されたセールスメールを生成するプロンプトを作成してください。

In [ ]:
from typing import List

def create_email_copy(name: str, age: int, interests: List[str], product: dict) -> str:
    # 以下のテンプレートを書き換えてください
    copy_msg = f'''
    {name}様に{product['name']}という商品を買ってもらうための宣伝メールを書いてください。
    ただし、{name}様は、{str(age)}歳で、{' と '.join(interests)}に関心を持っています。
    {product['name']}のセールスポイントは、{product['detail']}です。
    '''
    return copy_msg

# 商品情報
product = {
    'name': 'ジェットバイク',
    'detail': '空を自由に飛ぶことができるバイク'
}

# お客様情報
users = [
    {
        "name": "浦島太郎",
        "age": 56,
        "interests": ["釣り", "盆栽"]
    },
    {
        "name": "白雪姫",
        "age": 17,
        "interests": ["美容", "衣服", "ダンス"]
    }
]

for user in users:
    prompt = create_email_copy(**user, product=product)
    print('Prompt:\n', prompt)
    print('Output:\n', invoke_claude(prompt))

## 複数のプロンプトを組み合わせたシステムを構築する

### 基本的なチェーン

LLMシステムを開発する中で、LLMを多段で使ったり、他システムと連携したりする必要が出てきます。  
LangChain はこういった課題を解決するライブラリです。

ここでは、マーケティング支援アプリを例にとって、プロンプトをつなげていく処理を作成してみます。  
提供している製品の情報から企業名とサービス名を考案し、その情報をもとにスローガンを考えます。

まずはLangChainで使えるようにBedrockChat LLMを定義します。 

In [ ]:
llm = BedrockChat(
    client=bedrock_runtime,
    model_id="anthropic.claude-v2",
    model_kwargs={
        "temperature":0.0,
        "max_tokens_to_sample": 2000},
)

会社名を考えるChainを作成します。

In [ ]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="""
    <instruction>
    あなたはマーケティングの熟練者です。
    {product}を提供している会社について、独自性があり、誰にとっても覚えやすいような会社名を考えてください。
    ただし、会社名のみを出力し、それ以外のコメントは出力しないでください。
    </instruction>
    """
)

In [ ]:
# LLMChainの実装
company_name_chain = LLMChain(llm=llm, prompt=prompt, output_key="company_name")

In [ ]:
product = "スマートフォンゲーム"
company_name = company_name_chain.run(product)
print('output:', company_name)

また出力をバッチ的に一度に生成することも可能です

In [ ]:
qs = [
    {'product': "スポーツドリンク"},
    {'product': "運動靴"},
    {'product': "野球用グローブ"},
]

company_name_generated = company_name_chain.generate(qs)

In [ ]:
for response in company_name_generated.generations:
    print(response[0].text)

1つのプロンプトのみでは解決困難な、より現実的な問題を考えます。  
今回は、先ほど生成した会社名をもとに、開発可能なサービスに関するリストを出力します

In [ ]:
services_prompt = PromptTemplate(
    input_variables=["company_name", "product"],
    template="""{company_name}が開発可能な{product}に関するサービスのリストを出力してください。
    ただし、出力結果は, 区切りのリスト形式である必要があります。また、リストのみを出力しそれ以外の情報は出力しないでください。
    <example>(abc, def, ghi, jkl)</example>""",
)
services_chain = LLMChain(
    llm=llm,
    prompt=services_prompt,
    output_key="services"
)

In [ ]:
services = services_chain.run(product=product, company_name=company_name)
print(services)

次に企業のスローガンを作成します。

In [ ]:
slogan_prompt = PromptTemplate(
    input_variables=['product','company_name','services'],
    template="""
    <instruction>
    あなたはマーケティングの熟練者です。
    独自性が高く、かつ覚えやすいようなスローガンを考えてください。
    ただし、生成したスローガンのみを出力し、他のコメントや情報は出力しないようにしてください。
    </instruction>
    <context>
    企業名は{company_name}で{product}を販売しています。
    また、{services}といったサービスを提供しています。 
    </context>
    """
)
# Slogan chain
slogan_chain = LLMChain(
    llm=llm,
    prompt=slogan_prompt,
    output_key="slogan"
)

In [ ]:
slogan = slogan_chain.run(product=product, company_name=company_name, services=services)

In [ ]:
print(slogan)

### チェーンを接続する

ここまで、各プロンプトごとに入力・出力を定義し、必要な出力を得るためのプロンプトテンプレートを設計しました。  
次に、一連の動作を一括で行うようなチェーンを構築します。  

In [ ]:
marketing_chain = SequentialChain(
    chains=[company_name_chain, services_chain, slogan_chain],
    input_variables=['product'],
    output_variables=['company_name', 'services', 'slogan']
)

In [ ]:
marketing_chain("スマートフォンゲーム")

販売する製品を入力すると、会社名、サービス、会社のスローガンを考えてくれるチェーンができました。  

### 練習2

これまでの内容をもとに、オンラインマガジンの料理記事を作成できるチェーンを作成しましょう。  
チェーンのINPUTとして与えられる旅行先に関する情報をチェーンで拡充していきます。 

記事には以下の内容を含めてください。

* 旅行先の概要
* その地域でやるべきことのリスト
* 有名な郷土料理のレシピ（説明、材料のリスト、ステップバイステップの説明）

次の入力に基づいて記事を作成する必要があります。

* 旅行先（都市、国）

**余裕があれば考えてみましょう**  
* チェーンごとに出力のクオリティをチェックするような設計も検討してみましょう
* 結果を改善するための適切なエンジニアリング方法を考えてみましょう

In [ ]:
# 1. 目的地の概要を生成するチェーンを作成
overview_prompt = PromptTemplate(
    input_variables=["destination"],
    template="""{destination}の概要を教えてください。"""
)
overview_chain = LLMChain(llm=llm, prompt=overview_prompt, output_key="overview")

# 2. おすすめのアクティビティを生成するチェーンを作成  
activities_prompt = PromptTemplate(
    input_variables=["destination"],
    template="{destination}でできるアクティビティを教えてください。",
)
activities_chain = LLMChain(
    llm=llm,
    prompt=activities_prompt,
    output_key="activities"
)

# 3. その土地で有名な料理とレシピを生成するチェーンを作成
recipe_prompt = PromptTemplate(
    input_variables=['destination'],
    template="""
    {destination}で最も有名な料理の名前とレシピを教えてください。
    ただし、材料と作り方を含めてください。
    以下は解答例です。
    # 料理の名前
        - カレー
    # 材料
        - 牛肉
        - カレールウ
        - タマネギ
        - スパイス
    # 作り方
        1. タマネギを炒める 
        2. 牛肉を炒める
        3. 全て入れて熱する
    """
)
recipe_chain = LLMChain(
    llm=llm,
    prompt=recipe_prompt,
    output_key="recipe"
)


In [ ]:
from pprint import pprint

In [ ]:
magagine_chain = SequentialChain(
    chains=[overview_chain, activities_chain, recipe_chain],
    input_variables=['destination'],
    output_variables=['overview', 'activities', 'recipe']
)
pprint(magagine_chain('東京'))

旅行記事はできたでしょうか。  
ぜひプロンプトも変えて、試してみてください。  

## まとめ

この章では、基本的なプロンプトエンジニアリングと、LangChainによる抽象化について確認しました。  
モデルの利用方法、プロンプトテンプレートの作成、一連のステップの連鎖について学びました。  
プロンプトは Python のフォーマット機能で生成することもできますし、LangChain などのライブラリをうまく活用することで、より構造化された状態で生成することもできます。  